# Sequential chat example with math operator agents

From the [AutoGen _conversation patterns_ tutorial](https://microsoft.github.io/autogen/docs/tutorial/conversation-patterns#sequential-chats).

This is an example of a _sequential chat_:

> [A] sequence of chats between two agents, chained together by a mechanism called carryover, which brings the summary of the previous chat to the context of the next chat.

This examples uses a sequence of agents to get to a new number, given a starting number. For example, starting at 1, get to 13.

The agents are math operators and the carryover is the result of the previous operation.

See annotations in the cells for items to experiment with and other observations.

### LLM setup

In [1]:
# LLM setup
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### AutoGen setup

In [2]:
# This example is highly sensitive to the model you use
# It worked only with gpt-4 for me
# Change to gpt-3.5-turbo or gpt-4o to see different results
MODEL = "gpt-4"

CONFIG = [{"model": MODEL, "api_key": OPENAI_API_KEY}]

### Agents

The `number_agent` comes up with the initial and final number. The operator agents are math operators that can apply one operation at a time to the nummber they are given, resulting in a new number that is passed on to the next operator agent.

AutoGen will select the sequence of agents to get to the final number.

The `description` field is the important piece here: it is used to decide which agent to use next.

In [3]:
from autogen import ConversableAgent

# The Number Agent always returns the same numbers.
number_agent = ConversableAgent(
    name="Number_Agent",
    description="Return the numbers given.",
    system_message="You return me the numbers I give you, one number each line.",
    llm_config={"config_list": CONFIG},
    human_input_mode="NEVER",
)

# The Adder Agent adds 1 to each number it receives.
adder_agent = ConversableAgent(
    name="Adder_Agent",
    description="Add 1 to each input number.",
    system_message="You add 1 to each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": CONFIG},
    human_input_mode="NEVER",
)

# The Multiplier Agent multiplies each number it receives by 2.
multiplier_agent = ConversableAgent(
    name="Multiplier_Agent",
    description="Multiply each input number by 2.",
    system_message="You multiply each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": CONFIG},
    human_input_mode="NEVER",
)

# The Subtracter Agent subtracts 1 from each number it receives.
subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    description="Subtract 1 from each input number.",
    system_message="You subtract 1 from each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": CONFIG},
    human_input_mode="NEVER",
)

# The Divider Agent divides each number it receives by 2.
divider_agent = ConversableAgent(
    name="Divider_Agent",
    description="Divide each input number by 2.",
    system_message="You divide each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": CONFIG},
    human_input_mode="NEVER",
)

### Agent collaboration through a group chat

The agents are part of a group chat:

> The core idea of group chat is that all agents contribute to a single conversation thread and share the same context. This is useful for tasks that require collaboration among multiple agents.

"Collaboration" in this context means that one agent is select to perform an action. The result of this task is passed to the next agent selected, and so on until the goal is reached.

The secret sauce is the agent selection process. We are using `auto` here, which means that AutoGen will select the agents to use.

This is what happens behind the scenes (from `_auto_select_speaker` in AutoGen's `groupchat.py`):

> 1. Create a two-agent chat with a speaker selector agent and a speaker validator agent, like a nested chat
> 2. Inject the group messages into the new chat
> 3. Run the two-agent chat, evaluating the result of response from the speaker selector agent:
>    - If a single agent is provided then we return it and finish. If not, we add an additional message to this nested chat in an attempt to guide the LLM to a single agent response
> 4. Chat continues until a single agent is nominated or there are no more attempts left
> 5. If we run out of turns and no single agent can be determined, the next speaker in the list of agents is returned

In other words, AutoGen uses the LLM to select an agent. This process can consume a lot of tokens. We will see how many at the end of the notebook.  

In [4]:
from autogen import GroupChat

# This selection method uses the agent descriptions to select the agent
SPEAKER_SELECTION_METHOD = "auto"

group_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    speaker_selection_method="auto",
    messages=[],
    max_round=6,
)

In [5]:
from autogen import GroupChatManager

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": CONFIG},
)

### Running the example

Now that everything is set up, we can run the example. It wil log each step of the conversation.

Note that [AutoGen can cache LLM requests and responses](https://microsoft.github.io/autogen/docs/topics/llm-caching). If the examples runs very fast, it likely means the results have been cached and we are not going to the LLM for each step.

In [6]:
chat_result = number_agent.initiate_chat(
    group_chat_manager,
    message="My number is 3, I want to turn it into 13.",
    summary_method="reflection_with_llm",
)

Number_Agent (to chat_manager):

My number is 3, I want to turn it into 13.

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

6

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

26

--------------------------------------------------------------------------------

Next speaker: Subtracter_Agent

Subtracter_Agent (to chat_manager):

5
25

--------------------------------------------------------------------------------

Next speaker: Subtracter_Agent

Subtracter_Agent (to chat_manager):

5
25

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

6
26

--------------------------------------------------------------------------------


### Inspecting the results

#### Chat summary

The chat history shows each time an agent had to go to an LLM to complete at task. In our case, it was every time an operator agent was selected.

In [7]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': 'My number is 3, I want to turn it into 13.', 'role': 'assistant'},
 {'content': '6', 'name': 'Multiplier_Agent', 'role': 'user'},
 {'content': '26', 'name': 'Multiplier_Agent', 'role': 'user'},
 {'content': '5\n25', 'name': 'Subtracter_Agent', 'role': 'user'},
 {'content': '5\n25', 'name': 'Subtracter_Agent', 'role': 'user'},
 {'content': '6\n26', 'name': 'Adder_Agent', 'role': 'user'}]


#### Token consumption and costs

Note that [AutoGen can cache LLM requests and responses](https://microsoft.github.io/autogen/docs/topics/llm-caching). It reduces the actual number of LLM calls.

The cache is persistent. If you don't change the code, you should see only cached tokens used to complete the task. In addition, caching speeds up the process.

In [8]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-4-0613': {'completion_tokens': 77,
                                                     'cost': 0.00762,
                                                     'prompt_tokens': 100,
                                                     'total_tokens': 177},
                                      'total_cost': 0.00762},
 'usage_including_cached_inference': {'gpt-4-0613': {'completion_tokens': 77,
                                                     'cost': 0.00762,
                                                     'prompt_tokens': 100,
                                                     'total_tokens': 177},
                                      'total_cost': 0.00762}}


Is that a lot of tokens?

Yes. For comparison, [Kennedy's "we choose to go to the Moon" speech](https://en.wikipedia.org/wiki/We_choose_to_go_to_the_Moon) is about 350 tokens. So, this example uses about half of that to do a little math.


The famous part of the speech, for reference. It took him just over two minutes to deliver it.

> We set sail on this new sea because there is new knowledge to be gained, and new rights to be won, and they must be won and used for the progress of all people. For space science, like nuclear science and all technology, has no conscience of its own. Whether it will become a force for good or ill depends on man, and only if the United States occupies a position of pre-eminence can we help decide whether this new ocean will be a sea of peace or a new terrifying theater of war. I do not say that we should or will go unprotected against the hostile misuse of space any more than we go unprotected against the hostile use of land or sea, but I do say that space can be explored and mastered without feeding the fires of war, without repeating the mistakes that man has made in extending his writ around this globe of ours. There is no strife, no prejudice, no national conflict in outer space as yet. Its hazards are hostile to us all. Its conquest deserves the best of all mankind, and its opportunity for peaceful cooperation may never come again. But why, some say, the Moon? Why choose this as our goal? And they may well ask, why climb the highest mountain? Why, 35 years ago, fly the Atlantic? Why does Rice play Texas? We choose to go to the Moon. We choose to go to the Moon... We choose to go to the Moon in this decade and do the other things, not because they are easy, but because they are hard; because that goal will serve to organize and measure the best of our energies and skills, because that challenge is one that we are willing to accept, one we are unwilling to postpone, and one we intend to win, and the others, too.